In [1]:
# !which python

In [4]:
!pip install huggingface-hub

     |████████████████████████████████| 520 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 9.3 MB/s  eta 0:00:01
     |████████████████████████████████| 200 kB 27.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 26.8 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 174 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 21.1 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 28.0 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 16.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/sana/Desktop/github repossss/Datascience_Projects/copilot-agent-project/dementia_prediction/.venv/bin/python3 -m pip install --upgrad

# Hugging Face debug notebook (ask-my-health)

This notebook is for debugging Hugging Face connectivity *outside* the Streamlit app.

## What it does

1. Confirms your `HF_TOKEN` is valid (via `huggingface.co/api/whoami-v2`).
2. Probes a few likely Hugging Face Router endpoints for a **single model**.
3. Sends a **text-to-SQL prompt** and prints:
   - HTTP status code
   - response body (truncated)
   - parsed JSON (if any)

## Setup

Set your token in the same environment where you run Jupyter:

```bash
export HF_TOKEN="hf_..."
```



In [ ]:
import os
from huggingface_hub import InferenceClient

# Get token from environment variable - DO NOT hardcode tokens in notebooks!
# Set it before running: export HF_TOKEN='hf_your_token_here'
HF_TOKEN = os.getenv('HF_TOKEN')
if not HF_TOKEN:
    raise ValueError(
        "HF_TOKEN environment variable not set! "
        "Set it with: export HF_TOKEN='hf_your_token_here' "
        "Get your token from: https://huggingface.co/settings/tokens"
    )

if not HF_TOKEN.startswith('hf_'):
    raise ValueError(
        f"Invalid token format. Token must start with 'hf_'. "
        f"Your token starts with: '{HF_TOKEN[:3]}'"
    )

print(f"✓ Using HF_TOKEN from environment (starts with: {HF_TOKEN[:10]}...)")

client = InferenceClient(
    api_key=HF_TOKEN,
)

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3.2:novita",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='The capital of France is **Paris**. 🇫🇷', reasoning=None, tool_call_id=None, tool_calls=None)


In [8]:
schema = "daily_steps(date DATE, steps BIGINT)"
question = "How many steps did I walk this month?"

prompt = f"""
You are an expert at writing DuckDB SQL.

Return ONLY a single SQL query (no explanations, no markdown).
Rules:
- Must be a single SELECT statement.
- Query only the table {schema}.
- Do not use any other tables.
- If the question asks for a single number, return exactly one row with column alias answer.

Question: {question}
SQL:
""".strip()

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3.2:novita",
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
)

print(completion.choices[0].message.content)


SELECT COALESCE(SUM(steps), 0) AS answer FROM daily_steps WHERE date >= DATE_TRUNC('month', CURRENT_DATE) AND date < DATE_TRUNC('month', CURRENT_DATE) + INTERVAL 1 MONTH;
